In [1]:
# Import required libraries
import os
import json
import time
import datetime as dt
import csv
import pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Setup data directories
DATA_RAW = pathlib.Path("../data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {DATA_RAW.absolute()}")

# Load environment variables
load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded ALPHAVANTAGE_API_KEY?", bool(ALPHA_KEY))



Data directory: /Users/aman/Desktop/Bootcamp/bootcamp_aman_dhillon/project/notebooks/../data/raw
Loaded ALPHAVANTAGE_API_KEY? True


In [2]:
# Utility functions
def safe_stamp():
    """Generate a safe timestamp string"""
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

def safe_filename(prefix: str, meta: Dict[str, str]) -> str:
    """Generate a safe filename with metadata and timestamp"""
    mid = "_".join([f"{k}-{str(v).replace(' ', '-')[:20]}" for k, v in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

def validate_df(df: pd.DataFrame, required_cols: List[str], dtypes_map: Dict[str, str]) -> Dict[str, str]:
    """Validate DataFrame structure and data types"""
    msgs = {}
    
    # Check for missing columns
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing_cols'] = f"Missing columns: {missing}"
    
    # Validate data types
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    
    # Count NA values
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    
    return msgs



In [8]:
# Fixed version to handle the actual API response structure
SYMBOL = "TSLA"
print("Testing Alpha Vantage API for TSLA options...")
print("API Key loaded:", bool(ALPHA_KEY))

if ALPHA_KEY:
    print(f"\nFetching options data for {SYMBOL}...")
    
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "HISTORICAL_OPTIONS",
        "symbol": SYMBOL,
        "apikey": ALPHA_KEY
    }
    
    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        print(f"\nAPI Response received!")
        print(f"Response keys: {list(data.keys())}")
        
        if "data" in data and data["message"] == "success":
            print(f"✅ Success! Found options data")
            options_data = data["data"]
            print(f"Total contracts received: {len(options_data)}")
            
            # Convert to DataFrame
            df_options = pd.DataFrame(options_data)
            print(f"\nDataFrame created with shape: {df_options.shape}")
            print(f"Columns: {list(df_options.columns)}")
            
            # Show first few rows
            print(f"\nFirst few contracts:")
            print(df_options.head())
            
            # Check option types
            if 'type' in df_options.columns:
                print(f"\nOption types distribution:")
                print(df_options['type'].value_counts())
                
                # Separate by type
                df_calls = df_options[df_options['type'] == 'call'].copy()
                df_puts = df_options[df_options['type'] == 'put'].copy()
                
                print(f"\nCall options: {len(df_calls)}")
                print(f"Put options: {len(df_puts)}")
            
            # Show data types
            print(f"\nData types:")
            print(df_options.dtypes)
            
            # Show sample data
            print(f"\nSample call option:")
            if len(df_calls) > 0:
                print(df_calls.iloc[0])
                
        else:
            print(f"❌ API Error or unexpected response:")
            print(json.dumps(data, indent=2))
            
    except Exception as e:
        print(f"❌ Error: {e}")
        
else:
    print("❌ No API key found. Please set ALPHAVANTAGE_API_KEY in your .env file")

print("\n" + "="*50)
print("API Test Complete!")
print("="*50)

Testing Alpha Vantage API for TSLA options...
API Key loaded: True

Fetching options data for TSLA...

API Response received!
Response keys: ['endpoint', 'message', 'data']
✅ Success! Found options data
Total contracts received: 4256

DataFrame created with shape: (4256, 20)
Columns: ['contractID', 'symbol', 'expiration', 'strike', 'type', 'last', 'mark', 'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest', 'date', 'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho']

First few contracts:
            contractID symbol  expiration strike  type    last    mark  \
0  TSLA250822C00050000   TSLA  2025-08-22  50.00  call  273.12  274.07   
1  TSLA250822P00050000   TSLA  2025-08-22  50.00   put    0.00    0.01   
2  TSLA250822C00060000   TSLA  2025-08-22  60.00  call  249.38  264.05   
3  TSLA250822P00060000   TSLA  2025-08-22  60.00   put    0.00    0.01   
4  TSLA250822C00070000   TSLA  2025-08-22  70.00  call  272.61  254.05   

      bid bid_size     ask ask_size vo

In [9]:
print(df_options)

               contractID symbol  expiration  strike  type    last    mark  \
0     TSLA250822C00050000   TSLA  2025-08-22   50.00  call  273.12  274.07   
1     TSLA250822P00050000   TSLA  2025-08-22   50.00   put    0.00    0.01   
2     TSLA250822C00060000   TSLA  2025-08-22   60.00  call  249.38  264.05   
3     TSLA250822P00060000   TSLA  2025-08-22   60.00   put    0.00    0.01   
4     TSLA250822C00070000   TSLA  2025-08-22   70.00  call  272.61  254.05   
...                   ...    ...         ...     ...   ...     ...     ...   
4251  TSLA271217P00680000   TSLA  2027-12-17  680.00   put    0.00  368.38   
4252  TSLA271217C00690000   TSLA  2027-12-17  690.00  call   47.90   47.83   
4253  TSLA271217P00690000   TSLA  2027-12-17  690.00   put    0.00  377.30   
4254  TSLA271217C00700000   TSLA  2027-12-17  700.00  call   47.07   46.92   
4255  TSLA271217P00700000   TSLA  2027-12-17  700.00   put  389.00  386.52   

         bid bid_size     ask ask_size volume open_interest    

In [10]:
# Fixed version with date loop to accumulate data
SYMBOL = "TSLA"
print("Testing Alpha Vantage API for TSLA options...")
print("API Key loaded:", bool(ALPHA_KEY))

if ALPHA_KEY:
    print(f"\nFetching TSLA options data for multiple dates...")
    
    # Create date range from 2020 to today (adjust as needed)
    start_date = "2020-01-01"
    end_date = dt.datetime.now()
    
    # Generate list of dates to try (monthly intervals to avoid rate limits)
    dates_to_try = []
    current_date = dt.datetime.strptime(start_date, "%Y-%m-%d")
    
    while current_date <= end_date:
        dates_to_try.append(current_date.strftime("%Y-%m-%d"))
        current_date += dt.timedelta(days=30)  # Monthly intervals
    
    print(f"Will try {len(dates_to_try)} dates from {start_date} to {end_date.strftime('%Y-%m-%d')}")
    
    # Initialize empty DataFrame to accumulate all data
    df_options = pd.DataFrame()
    successful_dates = []
    failed_dates = []
    
    for i, date_str in enumerate(dates_to_try):
        print(f"\n[{i+1}/{len(dates_to_try)}] Trying date: {date_str}")
        
        url = "https://www.alphavantage.co/query"
        params = {
            "function": "HISTORICAL_OPTIONS",
            "symbol": SYMBOL,
            "date": date_str,
            "apikey": ALPHA_KEY
        }
        
        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            
            if "data" in data and data["message"] == "success":
                options_data = data["data"]
                print(f"✅ Success! Got {len(options_data)} contracts for {date_str}")
                
                # Convert to DataFrame and add date column
                df_temp = pd.DataFrame(options_data)
                df_temp['extraction_date'] = date_str  # Add the date we extracted from
                
                # Append to main DataFrame
                df_options = pd.concat([df_options, df_temp], ignore_index=True)
                successful_dates.append(date_str)
                
                print(f"   Total contracts so far: {len(df_options)}")
                
            else:
                print(f"❌ No data for {date_str}")
                if "Note" in data:
                    print(f"   Note: {data['Note']}")
                failed_dates.append(date_str)
            
            # Add delay to avoid rate limiting (Alpha Vantage has limits)
            time.sleep(1)  # 1 second delay between requests
            
        except Exception as e:
            print(f"❌ Error for {date_str}: {e}")
            failed_dates.append(date_str)
            time.sleep(1)
    
    # Summary of what we collected
    print("\n" + "="*60)
    print("EXTRACTION SUMMARY")
    print("="*60)
    print(f"Symbol: {SYMBOL}")
    print(f"Total contracts collected: {len(df_options)}")
    print(f"Successful dates: {len(successful_dates)}")
    print(f"Failed dates: {len(failed_dates)}")
    
    if len(df_options) > 0:
        print(f"\nDataFrame shape: {df_options.shape}")
        print(f"Columns: {list(df_options.columns)}")
        
        # Check option types
        if 'type' in df_options.columns:
            print(f"\nOption types distribution:")
            print(df_options['type'].value_counts())
            
            # Separate by type
            df_calls = df_options[df_options['type'] == 'call'].copy()
            df_puts = df_options[df_options['type'] == 'put'].copy()
            
            print(f"\nCall options: {len(df_calls)}")
            print(f"Put options: {len(df_puts)}")
        
        # Show date range
        if 'extraction_date' in df_options.columns:
            print(f"\nDate range covered:")
            print(f"Earliest: {df_options['extraction_date'].min()}")
            print(f"Latest: {df_options['extraction_date'].max()}")
        
        # Show first few rows
        print(f"\nFirst few contracts:")
        print(df_options.head())
        
        # Save the accumulated data
        fname = safe_filename(prefix="tsla_options_accumulated", meta={"symbol": SYMBOL, "dates": f"{len(successful_dates)}_dates"})
        out_path = DATA_RAW / fname
        df_options.to_csv(out_path, index=False)
        print(f"\n✅ Saved all accumulated data to: {out_path}")
        
    else:
        print("❌ No data was collected")
    
    # Show failed dates for debugging
    if failed_dates:
        print(f"\nFailed dates: {failed_dates}")
        
else:
    print("❌ No API key found. Please set ALPHAVANTAGE_API_KEY in your .env file")

print("\n" + "="*60)
print("EXTRACTION COMPLETE!")
print("="*60)

Testing Alpha Vantage API for TSLA options...
API Key loaded: True

Fetching TSLA options data for multiple dates...
Will try 69 dates from 2020-01-01 to 2025-08-21

[1/69] Trying date: 2020-01-01
❌ No data for 2020-01-01

[2/69] Trying date: 2020-01-31
✅ Success! Got 6626 contracts for 2020-01-31
   Total contracts so far: 6626

[3/69] Trying date: 2020-03-01
❌ No data for 2020-03-01

[4/69] Trying date: 2020-03-31
✅ Success! Got 7420 contracts for 2020-03-31
   Total contracts so far: 14046

[5/69] Trying date: 2020-04-30
✅ Success! Got 7394 contracts for 2020-04-30
   Total contracts so far: 21440

[6/69] Trying date: 2020-05-30
❌ No data for 2020-05-30

[7/69] Trying date: 2020-06-29
✅ Success! Got 6438 contracts for 2020-06-29
   Total contracts so far: 27878

[8/69] Trying date: 2020-07-29
✅ Success! Got 8432 contracts for 2020-07-29
   Total contracts so far: 36310

[9/69] Trying date: 2020-08-28
✅ Success! Got 8878 contracts for 2020-08-28
   Total contracts so far: 45188

[10/